In [1]:
from google.cloud import bigquery
bq = bigquery.Client(project="operating-pod-302903")

In [2]:
bq

In [3]:
bq.project

'operating-pod-302903'

In [4]:
no_cache = bigquery.QueryJobConfig(use_query_cache=False)

In [5]:
q = bq.query("""
SELECT geo_id, county_name
FROM bigquery-public-data.geo_us_boundaries.counties
WHERE county_name = 'Dane'
""", job_config=no_cache)
q.to_dataframe()

,geo_id,county_name
0,55025,Dane


In [6]:
#q1
q.to_dataframe().loc[0, 'geo_id']

'55025'

In [7]:
q2 = bq.query("""
SELECT state_fips_code, COUNT(*) as num_counties
FROM bigquery-public-data.geo_us_boundaries.counties
GROUP BY state_fips_code
""", job_config=no_cache)
q2.to_dataframe()

,state_fips_code,num_counties
0,01,67
1,02,29
2,04,15
3,05,75
4,06,58
5,08,64
6,09,8
7,10,3
8,11,1
9,12,67


In [8]:
ncdf = q2.to_dataframe()
ncdf = ncdf.sort_values(by = ['num_counties'], ascending=False).head(5)

In [9]:
#q2
d = {}
for i in ncdf.index:
    d[ncdf.loc[i, 'state_fips_code']] = ncdf.loc[i, 'num_counties']
    
d

{'48': 254, '13': 159, '51': 133, '21': 120, '29': 115}

In [10]:
q2.cache_hit

False

In [11]:
def charge(query):
    price = 5
    b = query.total_bytes_billed / 1024**4
    return b * price

In [12]:
#q3
prices = {'q1' : charge(q), 'q2': charge(q2)}
prices

{'q1': 4.76837158203125e-05, 'q2': 4.76837158203125e-05}

## Creating Dataset and loading hdma data

In [13]:
dataset = bigquery.Dataset("operating-pod-302903.p7")
bq.create_dataset(dataset, exists_ok=True)

Dataset(DatasetReference('operating-pod-302903', 'p7'))

In [14]:
config = bigquery.ExternalConfig("PARQUET")
config.source_uris = ["gs://project7_abhishek/hdma-wi-2021.parquet"]
# config.autodetect = True
table = bigquery.Table("operating-pod-302903.p7.hdma")
table.external_data_configuration = config
bq.create_table(table, exists_ok=True)

Table(TableReference(DatasetReference('operating-pod-302903', 'p7'), 'hdma'))

In [15]:
#q4
[ds.dataset_id for ds in bq.list_datasets("operating-pod-302903")]

['p7']

In [16]:
q5 = bq.query("""
SELECT pub.county_name, COUNT(*) AS num_loans
FROM operating-pod-302903.p7.hdma AS priv
INNER JOIN bigquery-public-data.geo_us_boundaries.counties AS pub
ON priv.county_code = pub.county_fips_code
GROUP BY pub.county_name
""", job_config=no_cache)
q5

QueryJob<project=operating-pod-302903, location=US, id=c3abc1d0-27a0-4828-be44-a3b7e7d603ed>

In [17]:
df5 = q5.to_dataframe()
df5

,county_name,num_loans
0,Dodge,5054
1,Adams,1491
2,Burnett,1386
3,Kenosha,10744
4,Outagamie,11523
...,...,...
67,Rusk,605
68,Menominee,144
69,Forest,517
70,Buffalo,532


In [18]:
topCounties = df5.sort_values(by = ['num_loans'], ascending=False).head(10)
topCounties

,county_name,num_loans
12,Milwaukee,46570
14,Dane,38557
15,Waukesha,34159
13,Brown,15615
8,Racine,13007
4,Outagamie,11523
3,Kenosha,10744
20,Washington,10726
40,Rock,9834
23,Winnebago,9310


In [19]:
#q5
d5 = {}
for i in topCounties.index:
    d5[topCounties.loc[i, 'county_name']] = topCounties.loc[i, 'num_loans']
    
d5

{'Milwaukee': 46570,
 'Dane': 38557,
 'Waukesha': 34159,
 'Brown': 15615,
 'Racine': 13007,
 'Outagamie': 11523,
 'Kenosha': 10744,
 'Washington': 10726,
 'Rock': 9834,
 'Winnebago': 9310}

## Loading data from google form

In [20]:
url = "https://drive.google.com/open?id=1e2qLPyxZ7s5ibMyEg7bxX2wWYAQD2ROAUcZuv8fqhnA"

config = bigquery.ExternalConfig("GOOGLE_SHEETS")
config.source_uris = [url]
config.autodetect = True
table = bigquery.Table("operating-pod-302903.p7.applications")
table.external_data_configuration = config
bq.create_table(table, exists_ok=True)

Table(TableReference(DatasetReference('operating-pod-302903', 'p7'), 'applications'))

In [21]:
q6 = bq.query("""
SELECT COUNT(*) as cnt
FROM operating-pod-302903.p7.applications
WHERE INCOME = 100000
""", job_config=no_cache)
df6 = q6.to_dataframe()
df6

,cnt
0,12


In [22]:
#q6
df6.loc[0, 'cnt']

12

In [23]:
q7 = bq.query("""
SELECT pub.county_name, COUNT(*) as cnt
FROM operating-pod-302903.p7.applications AS apps
INNER JOIN bigquery-public-data.geo_us_boundaries.counties AS pub
ON (ST_WITHIN(ST_GEOGPOINT(apps.longitude, apps.latitude), pub.county_geom))
WHERE pub.state_fips_code = '55' and apps.latitude <= 90 and apps.latitude >= -90 and apps.longitude <= 180 and apps.longitude >= -180
GROUP BY pub.county_name
""", job_config=no_cache)
df7 = q7.to_dataframe()
df7

,county_name,cnt
0,Dane,27
1,Oneida,1
2,Door,5
3,Iron,1
4,Jefferson,2
5,Walworth,2
6,Monroe,1
7,Barron,1
8,Ashland,1
9,Waukesha,1


In [24]:
#q7
d7 = {}
for i in range(len(df7)):
    d7[df7.loc[i, 'county_name']] = df7.loc[i, 'cnt']
    
d7

{'Dane': 27,
 'Oneida': 1,
 'Door': 5,
 'Iron': 1,
 'Jefferson': 2,
 'Walworth': 2,
 'Monroe': 1,
 'Barron': 1,
 'Ashland': 1,
 'Waukesha': 1,
 'Brown': 3,
 'Green Lake': 1,
 'Adams': 1,
 'Bayfield': 2,
 'Sauk': 1,
 'Columbia': 2,
 'La Crosse': 1,
 'Iowa': 1,
 'Sheboygan': 1,
 'Kewaunee': 1,
 'Milwaukee': 1}

In [25]:
x = bq.query("""
SELECT *
FROM operating-pod-302903.p7.applications
limit 10
""", job_config=no_cache)
x = x.to_dataframe()
x

,Timestamp,loan_amount,income,latitude,longitude
0,2023-04-18 15:47:56.141000+00:00,200000,75000,43.069610,-89.411429
1,2023-04-18 22:02:45.381000+00:00,500000,125000,43.044469,-89.432674
2,2023-04-19 14:24:42.341000+00:00,350000,80000,44.513755,-88.057333
3,2023-04-20 08:59:51.179000+00:00,190000,150000,43.889848,-91.197627
4,2023-04-25 22:12:49.061000+00:00,250000,25000,43.905450,-89.060410
5,2023-04-25 22:16:03.936000+00:00,1200000,75000,42.711962,-88.639941
6,2023-04-26 17:29:54.788000+00:00,10000000,10000000,43.072125,-89.403310
7,2023-04-27 19:01:46.396000+00:00,10000,1,43.072200,89.400800
8,2023-04-27 20:06:10.639000+00:00,250000,150000,45.111400,87.047100
9,2023-04-27 20:40:45.871000+00:00,200000,150000,45.031929,-87.222494


## Machine Learning

In [26]:
%load_ext google.cloud.bigquery

In [27]:
%%bigquery
CREATE OR REPLACE TABLE operating-pod-302903.p7.traintest

AS 

SELECT income, loan_term, loan_amount, rand() < 0.25 AS test
FROM operating-pod-302903.p7.hdma

Query is running:   0%|          |

""


In [28]:
%%bigquery
CREATE OR REPLACE MODEL `operating-pod-302903.p7.mymodel`
OPTIONS(model_type='LINEAR_REG', INPUT_LABEL_COLS=['loan_amount'])

AS

SELECT income, loan_term, loan_amount
FROM `operating-pod-302903.p7.traintest`
WHERE NOT test

Query is running:   0%|          |

""


In [29]:
weights = bq.query("""
SELECT * FROM
ML.WEIGHTS(MODEL `operating-pod-302903.p7.mymodel`)
""", job_config=no_cache)
weights_df = weights.to_dataframe()
weights_df

,processed_input,weight,category_weights
0,income,0.798789,[]
1,loan_term,381.032894,[]
2,__INTERCEPT__,-330.773732,[]


In [30]:
eval = bq.query("""
SELECT *
FROM
ML.EVALUATE(
    MODEL `operating-pod-302903.p7.mymodel`,
    (
        SELECT income, loan_term, loan_amount
        FROM `operating-pod-302903.p7.traintest`
        WHERE test
    )
)
""", job_config=no_cache)
eval_df = eval.to_dataframe()
eval_df

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,76918.449483,1.292503e+10,0.425745,55390.188549,0.298104,0.298105


In [31]:
#q8
eval_df.iloc[0].at["r2_score"]

0.2981038427285001

In [32]:
#q9
weights_df.loc[weights_df.processed_input == 'income']['weight'].iloc[0]

0.79878858470684

In [33]:
preds = bq.query("""
SELECT *
FROM
ML.PREDICT(
    MODEL `operating-pod-302903.p7.mymodel`,
    (
        SELECT income, 360 AS loan_term, loan_amount
        FROM `operating-pod-302903.p7.applications`
    )
)
""", job_config=no_cache)
preds_df = preds.to_dataframe()
preds_df

,predicted_loan_amount,income,loan_term,loan_amount
0,196750.211844,75000,360,200000
1,236689.641080,125000,360,500000
2,200744.154768,80000,360,350000
3,256659.355697,150000,360,190000
4,156810.782609,25000,360,250000
...,...,...,...,...
100,216719.926462,100000,360,599
101,256659.355697,150000,360,700000
102,138704.641759,2333,360,2333
103,456356.501874,400000,360,1200000


In [34]:
#q10
percent_larger = len(preds_df.loc[preds_df.predicted_loan_amount < preds_df.loan_amount])
total = len(preds_df)
percent_larger/total

0.6